In [12]:
#!pip install spotipy --upgrade
%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
import json

with open("../secrets.json", "r") as f:
    secrets = json.load(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2


CLIENT_ID = secrets["spotify_client_id"]
CLIENT_SECRET = secrets["spotify_client_secret"]

credentials = oauth2.SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)

# track = "coldplay yellow"
# res = spotify.search(track, type="track", market="US", limit=1)
# print(res)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# print(json.dumps(res, indent=4, sort_keys=True))

<IPython.core.display.Javascript object>

In [6]:
# res = sp.featured_playlists()
# print(response["message"])

# while response:
#     playlists = response["playlists"]
#     for i, item in enumerate(playlists["items"]):
#         print(playlists["offset"] + i, item["name"])

#     if playlists["next"]:
#         response = sp.next(playlists)
#     else:
#         response = None

<IPython.core.display.Javascript object>

In [7]:
# print(json.dumps(res, indent=4, sort_keys=True))

<IPython.core.display.Javascript object>

Let's think about making a simple goal. How do I get my song onto running playlists?

In [8]:
# first get a list of running playlists from spotify using the search function
s_limit = 10
res = sp.search("Running", limit=s_limit, type="playlist", market="US")


<IPython.core.display.Javascript object>

``` python
playlist_info = {}
max_results = 30
offset = 0
while offset < max_results:
    playlists = res["playlists"]
    for i, item in enumerate(playlists["items"]):
        # Store the name as a key. Get the number of tracks and uri
        playlist_info[item["name"]] = [item["tracks"]["total"], item["uri"]]
    if playlists["next"]:
        res = sp.next(playlists)
        offset += s_limit
    else:
        res = None
playlist_info
```

In [9]:
# pd.DataFrame(playlist_info).T.to_csv("playlists_taken_from.csv")

<IPython.core.display.Javascript object>

In [10]:
# len(playlist_info)

<IPython.core.display.Javascript object>

In [11]:
# curr_playlist = list(playlist_info.items())[0]
# curr_playlist[1][1]

<IPython.core.display.Javascript object>

``` python
from pprint import pprint
#curr_playlist = list(playlist_info.items())[0]
all_track_ids = []

for p in list(playlist_info.items()):

    pl_id = p[1][1]
    offset = 0

    track_ids = []

    while True:
        response = sp.playlist_tracks(pl_id,
                                     offset=offset,
                                     fields='items.track.id,total',
                                     additional_types=['track'])
        #store the track ids from the playlist
        for item in response['items']:
            track_ids.append( item['track']['id'])

        offset = offset + len(response['items'])
        print(offset, "/", response['total'])

        if len(response['items']) == 0:
            break
            
    #Add the per_playlist ids to the master list
    all_track_ids.extend(track_ids)
```

In [12]:
# len(all_track_ids)

<IPython.core.display.Javascript object>

In [13]:
# all_track_ids_np = np.array(all_track_ids)

<IPython.core.display.Javascript object>

In [14]:
# all_track_ids_np = np.unique(all_track_ids_np)

<IPython.core.display.Javascript object>

In [15]:
# len(all_track_ids_np)

<IPython.core.display.Javascript object>

In [16]:
# all_track_ids_np

<IPython.core.display.Javascript object>

In [17]:
# pd.DataFrame(all_track_ids_np).to_csv("running_playlists_track_ids.csv")

<IPython.core.display.Javascript object>

In [16]:
# track_ids = pd.read_csv("running_playlists_track_ids.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
# track_features = []


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
# tracks = list(track_ids["0"])[:49]
# response = sp.tracks(tracks)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Get the info for all tracks in the db and store each batch inside of a list
``` python
track_info = []

import time

st = 0
end = len(list(track_ids["0"]))
# end = 200
step = 50

list(range(st, end, step))
for i in range(st, end, step):
    print(i)
    if len(list(track_ids["0"])) - i >= step - 1:
        response = sp.tracks(list(track_ids["0"][i : i + step]))
    else:
        response = sp.tracks(list(track_ids["0"][i:]))
    time.sleep(2)
    track_info.append(response)
 ```

Iterate through each batch of songs and create a dictionary with ids as the key
```python
info_dict = {}
for ind in track_info[0]["tracks"]:
    track_artist = ind["artists"][0]["name"]

    track_name = ind["name"]

    track_album = ind["album"]["name"]

    track_popularity = ind["popularity"]

    track_id = ind["id"]

    info_dict[track_id] = [track_artist, track_name, track_album, track_popularity]
info_dict
```

```python
info_dict = {}
for batch in track_info:
    type(batch)
    for ind in batch["tracks"]:
        track_artist = ind["artists"][0]["name"]

        track_name = ind["name"]

        track_album = ind["album"]["name"]

        track_popularity = ind["popularity"]

        track_id = ind["id"]

        info_dict[track_id] = [track_artist, track_name, track_album, track_popularity]
len(info_dict)
```

convert to a dataframe, transpose, and rename columns
```python
tdf = pd.DataFrame(info_dict).T
tdf = tdf.reset_index()
# tdf.columns

tdf = tdf.rename(
    columns={"index": "id", 0: "artist", 1: "title", 2: "album", 3: "popularity"}
)
tdf.head()
``` 

In [131]:
#tdf.to_csv("track_info.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True))

Get all the audio features for each track in batches of 100
 ``` python
track_features = []

import time

st = 0
end = len(list(track_ids["0"]))
step = 100

list(range(st, end, step))
for i in range(st, end, step):
    print(i)
    if len(list(track_ids["0"])) - i >= 100:
        features_response = sp.audio_features(list(track_ids["0"][i : i + step]))
    else:
        features_response = sp.audio_features(list(track_ids["0"][i:]))
    time.sleep(2)
    track_features.append(features_response)

 ```

Iterate through, convert each to a dataframe, and concatenate the whole thing
 ``` python
dfs = []
for item in track_features:
    df = pd.DataFrame.from_dict(item)
    dfs.append(df)

track_features = pd.concat(dfs)
```

In [134]:
#pd.DataFrame(track_features).to_csv("track_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Clean and join the two dfs
```python
track_features = pd.read_csv("track_features.csv")
track_info = pd.read_csv("track_info.csv")

track_info.drop(columns="Unnamed: 0", inplace=True)
track_features.drop(columns="Unnamed: 0", inplace=True)

track_features.drop(columns=["track_href", "analysis_url", "uri", "type"], inplace=True)

full_df = track_info.merge(track_features, on="id", how="outer")

full_df.head()
```

In [153]:
#full_df.to_csv("running_playlist_tracks.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [154]:
running = pd.read_csv("running_playlist_tracks.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [157]:
running.drop(columns="Unnamed: 0", inplace=True)
running.head()

,id,artist,title,album,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,00GOPLxW4PGQuUYdPJh8K1,Childish Gambino,IV. Sweatpants,Because the Internet,71,0.710,0.470,1,-9.330,0,0.1420,0.15400,0.00000,0.6430,0.581,80.027,180675,4
1,00QyLmjxaSEE8qIZQjBXBj,2 Chainz,We Own It (Fast & Furious),Fast & Furious 6,62,0.554,0.899,8,-4.573,1,0.4080,0.05210,0.00000,0.0568,0.552,171.966,227893,4
2,00V7a8gzdZQgvulLe3MDe3,Elliphant,To The End,To The End,0,0.650,0.893,11,-3.340,0,0.0426,0.01300,0.00384,0.0898,0.246,115.005,259773,4
3,00ko9WaS4jOX1kEk3gvHjf,Chance the Rapper,Hot Shower,The Big Day,74,0.899,0.509,9,-8.040,1,0.3870,0.00157,0.00000,0.0573,0.599,150.001,225924,4
4,00qOE7OjRl0BpYiCiweZB2,Foreigner,Juke Box Hero,4 (Expanded),68,0.357,0.653,9,-5.554,1,0.0654,0.08280,0.00000,0.0844,0.522,176.647,259800,4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>